# Networks

In [1]:
!pip install networkx ipysigma pelote

  Using cached pelote-0.8.1-py3-none-any.whl (39 kB)
  Using cached llist-0.7.1.tar.gz (42 kB)
  Preparing metadata (setup.py) ... done
  DEPRECATION: llist is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for llist ... done


In [5]:
import networkx as nx
import csv
from ipysigma import Sigma

In [3]:
Sigma(nx.karate_club_graph())

Sigma(nx.Graph with 34 nodes and 78 edges)

## Learning how to create a graph

https://networkx.org/

In [5]:
graph = nx.Graph()
type(graph)

networkx.classes.graph.Graph

In [6]:
graph.add_node('one')

In [7]:
len(graph)

1

In [8]:
graph.add_node('two')
graph.add_node('three')

In [9]:
len(graph)

3

In [10]:
graph.add_edge('one', 'two')

In [11]:
graph.order(), graph.size()

(3, 1)

In [12]:
graph.add_edge('two', 'three')

In [14]:
Sigma(graph)

Sigma(nx.Graph with 3 nodes and 2 edges)

In [15]:
graph.add_node('four', color='red')

In [16]:
# Iterating over nodes
for node in graph:
    print(node)

one
two
three
four


In [18]:
# Iterating over nodes with attributes
for node, attributes in graph.nodes.data():
    print(node, attributes)

one {}
two {}
three {}
four {'color': 'red'}


In [19]:
# Iterate over edges
for source, target in graph.edges:
    print(source, target)

one two
two three


In [20]:
# Iterate over edges with attributes
for source, target, attributes in graph.edges.data():
    print(source, target, attributes)

one two {}
two three {}


In [21]:
'five' in graph

False

In [23]:
graph.has_edge('one', 'two'), graph.has_edge('two', 'one')

(True, True)

In [24]:
digraph = nx.DiGraph()

## A bipartite graph of twitter accounts to hashtags

### Collecting the data

In [6]:
TWEETS = []

with open('/home/yomguithereal/Downloads/sciencespo-blocage-tweets.csv', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    
    for row in reader:
        if row['hashtags']:
            row['hashtags'] = row['hashtags'].split('|')
        else:
            row['hashtags'] = []
            
        row['user_followers'] = int(row['user_followers'])
            
        TWEETS.append(row)

In [29]:
tweets_with_hashtags = 0

for tweet in TWEETS:
    if tweet['hashtags']:
        tweets_with_hashtags += 1
        
tweets_with_hashtags

554

### Building the network

In [34]:
bipartite = nx.Graph()

for tweet in TWEETS:
    if not tweet['hashtags']:
        continue
    
    user = '@' + tweet['user_screen_name']
    
    if user not in bipartite:
        bipartite.add_node(user, part='user', followers=tweet['user_followers'])
        
    for hashtag in tweet['hashtags']:
        hashtag = '#' + hashtag
        
        if hashtag not in bipartite:
            bipartite.add_node(hashtag, part='hashtag')
        
        bipartite.add_edge(user, hashtag)
        
bipartite.order(), bipartite.size()

(849, 1090)

In [31]:
from pelote import crop_to_largest_connected_component

In [40]:
crop_to_largest_connected_component(bipartite)

In [33]:
Sigma(bipartite, node_size=bipartite.degree, node_color='part', node_color_palette={'user': 'red', 'hashtag': 'blue'})

Sigma(nx.Graph with 691 nodes and 996 edges)

In [20]:
Sigma(
    bipartite,
    node_size=bipartite.degree,
    node_metrics=['louvain'],
    node_color='louvain'
)

Sigma(nx.Graph with 691 nodes and 996 edges)

## Monopartite projections

In [21]:
from pelote import monopartite_projection

In [41]:
user_monopartite = monopartite_projection(bipartite, 'user')

In [42]:
user_monopartite.order(), user_monopartite.size(), nx.density(user_monopartite)

(359, 16570, 0.25785468635719955)

In [51]:
Sigma(
    user_monopartite,
    node_size='followers',
    node_size_scale=('sqrt', 2.5),
    node_metrics=['louvain'],
    node_color='louvain'
)

Sigma(nx.Graph with 359 nodes and 16,570 edges)

In [57]:
user_monopartite_by_jaccard = monopartite_projection(bipartite, 'user', metric='jaccard')

In [58]:
Sigma(user_monopartite_by_jaccard)

Sigma(nx.Graph with 359 nodes and 16,570 edges)

In [52]:
hashtag_monopartite = monopartite_projection(bipartite, 'hashtag')

In [53]:
hashtag_monopartite.order(), hashtag_monopartite.size(), nx.density(hashtag_monopartite)

(332, 1790, 0.03257743966803771)

In [56]:
Sigma(
    hashtag_monopartite,
    node_size=hashtag_monopartite.degree,
    node_metrics=['louvain'],
    node_color='louvain'
)

Sigma(nx.Graph with 332 nodes and 1,790 edges)